In [1]:
import json
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

def read_financial_json(file_path: str) -> str:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return json.dumps(data, indent=2)

system_prompt = """
You are a **financial data analyst** that extracts and transforms banking data into structured financial profiles.
You are given with:
1. The **FinanceProfile schema** — this schema defines fixed keys for long-term user financial tracking.
2. The **user’s latest financial data**.

Your objectives:
- Start with the **exact FinanceProfile schema provided below**. All keys in this schema must always be present in the final output, in the **same order** as given.
- Populate or update fields **only** when data is explicitly present or can be inferred with **high confidence** from the input data.
- Perform **all necessary calculations**.
- If relevant information exists that is not covered by the schema but would be valuable for **long-term financial planning**. you **must** add these as new fields — appended **after** the original schema keys. These new fields must use **snake_case** and have descriptive names.
- Any missing or uncertain values must remain `null`.
- Dates must follow **ISO 8601** format.
- Output must be valid JSON with exactly **three** top-level keys:
  1. `"FinanceProfile"` — schema fields (with added fields if applicable).
  2. `"calculations"` — a detailed explanation of how each non-null field was computed or inferred.
  3. `"profile_summary"` — a single well-written paragraph in plain English summarizing the user’s financial situation, key goals, spending habits, debts, and overall profile.

---

### FinanceProfile Schema:
```json
{{
  "user_age_years": null,
  "user_occupation_job": null,
  "user_location_city_country": null,
  "user_has_spouse_partner": null,
  "user_number_of_children": null,
  "income_total_yearly_amount": null,
  "income_monthly_take_home": null,
  "income_from_salary": null,
  "income_from_business": null,
  "income_from_investments": null,
  "income_from_rental": null,
  "income_other_sources": null,
  "expense_total_monthly": null,
  "expense_housing_rent_mortgage": null,
  "expense_food_groceries_dining": null,
  "expense_transportation_car_gas": null,
  "expense_utilities_bills": null,
  "expense_entertainment_hobbies": null,
  "expense_other_categories": null,
  "savings_monthly_amount": null,
  "savings_emergency_fund_total": null,
  "savings_percentage_of_income": null,
  "goal_buy_house_amount_needed": null,
  "goal_buy_house_timeline_years": null,
  "goal_retirement_target_amount": null,
  "goal_retirement_age_target": null,
  "goal_children_education_amount": null,
  "goal_vacation_travel_budget": null,
  "goal_other_major_purchases": null,
  "goal_most_important_priority": null,
  "investment_total_portfolio_value": null,
  "investment_stocks_value": null,
  "investment_bonds_value": null,
  "investment_mutual_funds_value": null,
  "investment_etf_value": null,
  "investment_crypto_value": null,
  "investment_real_estate_value": null,
  "investment_gold_precious_metals": null,
  "investment_fixed_deposits_value": null,
  "investment_retirement_401k_ira": null,
  "debt_total_amount_owed": null,
  "debt_home_loan_mortgage": null,
  "debt_car_loan_amount": null,
  "debt_student_loan_amount": null,
  "debt_credit_card_balance": null,
  "debt_personal_loan_amount": null,
  "debt_monthly_total_payments": null,
  "risk_tolerance_level": null,
  "risk_can_afford_to_lose_amount": null,
  "risk_investment_experience_years": null,
  "risk_reaction_to_market_drop": null,
  "prefer_stocks_over_bonds": null,
  "prefer_domestic_over_international": null,
  "prefer_growth_over_dividend": null,
  "prefer_individual_stocks_over_funds": null,
  "prefer_active_over_passive": null,
  "interested_in_crypto": null,
  "interested_in_real_estate": null,
  "avoid_specific_sectors": null,
  "tax_bracket_percentage": null,
  "tax_filing_status": null,
  "tax_state_residence": null,
  "tax_deductions_claimed": null,
  "insurance_has_life": null,
  "insurance_has_health": null,
  "insurance_has_disability": null,
  "insurance_has_home_renters": null,
  "insurance_has_auto": null,
  "knowledge_understands_stocks": null,
  "knowledge_understands_bonds": null,
  "knowledge_understands_mutual_funds": null,
  "knowledge_needs_basic_education": null,
  "knowledge_confident_level": null,
  "history_best_investment_made": null,
  "history_worst_investment_made": null,
  "history_learned_lessons": null,
  "history_years_investing": null,
  "plans_major_purchase_next_year": null,
  "plans_career_change_expected": null,
  "plans_expecting_inheritance": null,
  "plans_starting_business": null,
  "prefers_simple_explanations": null,
  "prefers_detailed_analysis": null,
  "prefers_conservative_advice": null,
  "prefers_aggressive_strategies": null,
  "profile_last_updated_date": null,
  "portfolio_last_reviewed_date": null,
  "goals_last_discussed_date": null,
  "notes_special_circumstances": null,
  "notes_specific_questions_asked": null,
  "notes_advice_given_previously": null
}}

User Data:
{input_json}

Final Output Format:

{{
  "FinanceProfile": {{ ... }},
  "calculations": {{ ... }},
  "profile_summary": "..."
}}
"""



In [3]:
prompt = PromptTemplate(
input_variables=["input_json"],
template=system_prompt
)
json_data_str = read_financial_json("data.json")
final_prompt = prompt.format(input_json=json_data_str)
print(f"Final Prompt: {final_prompt}")
llm = ChatGoogleGenerativeAI(
model="gemini-2.0-flash",
temperature=0.4,
)


response = llm.invoke(final_prompt)

print(response.content)

Final Prompt: 
You are a **financial data analyst** that extracts and transforms banking data into structured financial profiles.
You are given with:
1. The **FinanceProfile schema** — this schema defines fixed keys for long-term user financial tracking.
2. The **user’s latest financial data**.

Your objectives:
- Start with the **exact FinanceProfile schema provided below**. All keys in this schema must always be present in the final output, in the **same order** as given.
- Populate or update fields **only** when data is explicitly present or can be inferred with **high confidence** from the input data.
- Perform **all necessary calculations**.
- If relevant information exists that is not covered by the schema but would be valuable for **long-term financial planning**. you **must** add these as new fields — appended **after** the original schema keys. These new fields must use **snake_case** and have descriptive names.
- Any missing or uncertain values must remain `null`.
- Dates mus

In [9]:
suggestions_prompt = """
You are a **professional financial advisor** who provides personalized recommendations based on a user’s long-term financial profile and their current situation.

You are given:
1. **FinanceProfile** — a structured JSON object with the user’s detailed long-term financial data.
2. **Profile Summary** — a short narrative summarizing the user’s situation.

Your task:
- Use the given data to create a **personalized recommendation** as if you were speaking directly to the user.
- Do **not** repeat back all the facts they already know — instead, focus on **what they should do next** to improve their financial health.
- Make the advice **specific to them** based on their profile, debt levels, savings, income, location, and goals.
- Use a **supportive, encouraging, and human tone** that makes them feel understood and motivated.
- Focus on **clear, practical steps** they can take, including:
  - Strategies to grow income.
  - Smart ways to manage and optimize expenses.
  - How to build or strengthen their emergency fund.
  - The best way to handle debts in their situation.
  - Opportunities to start or improve investments.
  - Long-term planning for goals like buying a home, retirement, or education.
- Avoid sounding robotic or overly formal keep it conversational.
- Write as **one cohesive paragraph** that feels like a friendly one-on-one chat.
---

Input:
{
  "FinanceProfile": {
    "user_age_years": 33,
    "user_occupation_job": "Account Manager",
    "user_location_city_country": "Ahmedabad, India",
    "user_has_spouse_partner": null,
    "user_number_of_children": null,
    "income_total_yearly_amount": 1400000.0,
    "income_monthly_take_home": 116666.67,
    "income_from_salary": null,
    "income_from_business": null,
    "income_from_investments": null,
    "income_from_rental": null,
    "income_other_sources": null,
    "expense_total_monthly": 3825.0,
    "expense_housing_rent_mortgage": null,
    "expense_food_groceries_dining": null,
    "expense_transportation_car_gas": null,
    "expense_utilities_bills": null,
    "expense_entertainment_hobbies": null,
    "expense_other_categories": null,
    "savings_monthly_amount": null,
    "savings_emergency_fund_total": null,
    "savings_percentage_of_income": null,
    "goal_buy_house_amount_needed": null,
    "goal_buy_house_timeline_years": null,
    "goal_retirement_target_amount": null,
    "goal_retirement_age_target": null,
    "goal_children_education_amount": null,
    "goal_vacation_travel_budget": null,
    "goal_other_major_purchases": null,
    "goal_most_important_priority": null,
    "investment_total_portfolio_value": null,
    "investment_stocks_value": null,
    "investment_bonds_value": null,
    "investment_mutual_funds_value": null,
    "investment_etf_value": null,
    "investment_crypto_value": null,
    "investment_real_estate_value": null,
    "investment_gold_precious_metals": null,
    "investment_fixed_deposits_value": null,
    "investment_retirement_401k_ira": null,
    "debt_total_amount_owed": 1150000.0,
    "debt_home_loan_mortgage": null,
    "debt_car_loan_amount": 1200000.0,
    "debt_student_loan_amount": null,
    "debt_credit_card_balance": null,
    "debt_personal_loan_amount": null,
    "debt_monthly_total_payments": 25000.0,
    "risk_tolerance_level": null,
    "risk_can_afford_to_lose_amount": null,
    "risk_investment_experience_years": null,
    "risk_reaction_to_market_drop": null,
    "prefer_stocks_over_bonds": null,
    "prefer_domestic_over_international": null,
    "prefer_growth_over_dividend": null,
    "prefer_individual_stocks_over_funds": null,
    "prefer_active_over_passive": null,
    "interested_in_crypto": null,
    "interested_in_real_estate": null,
    "avoid_specific_sectors": null,
    "tax_bracket_percentage": null,
    "tax_filing_status": null,
    "tax_state_residence": "Gujarat",
    "tax_deductions_claimed": null,
    "insurance_has_life": null,
    "insurance_has_health": null,
    "insurance_has_disability": null,
    "insurance_has_home_renters": null,
    "insurance_has_auto": null,
    "knowledge_understands_stocks": null,
    "knowledge_understands_bonds": null,
    "knowledge_understands_mutual_funds": null,
    "knowledge_needs_basic_education": null,
    "knowledge_confident_level": null,
    "history_best_investment_made": null,
    "history_worst_investment_made": null,
    "history_learned_lessons": null,
    "history_years_investing": null,
    "plans_major_purchase_next_year": null,
    "plans_career_change_expected": null,
    "plans_expecting_inheritance": null,
    "plans_starting_business": null,
    "prefers_simple_explanations": null,
    "prefers_detailed_analysis": null,
    "prefers_conservative_advice": null,
    "prefers_aggressive_strategies": null,
    "profile_last_updated_date": null,
    "portfolio_last_reviewed_date": null,
    "goals_last_discussed_date": null,
    "notes_special_circumstances": null,
    "notes_specific_questions_asked": null,
    "notes_advice_given_previously": null,
    "monthly_loan_payment": 25000.0,
    "available_credit": 125000.75
  }
  }

Profile Summary:
{  
  "profile_summary": "Neha Patel is a 33-year-old Account Manager residing in Ahmedabad, India. Her estimated annual income is approximately ₹1,400,000, with a monthly take-home of around ₹116,666.67. Her monthly expenses are estimated to be ₹3,825. She has an active car loan with a total amount owed of approximately ₹1,150,000 and makes monthly payments of ₹25,000. She has a current account with a balance of ₹125,000.75. There is no information available regarding her savings, investments, or specific financial goals. Further information is needed to create a comprehensive financial plan."

}

Output:
"""


In [10]:

llm = ChatGoogleGenerativeAI(
model="gemini-2.0-flash",
temperature=0.4,
)
response = llm.invoke(suggestions_prompt)

print(response.content)

Okay, Neha, it's great to see you're proactive about your financial future! With a solid income like yours in Ahmedabad, we can definitely work towards building a strong financial foundation. The first thing I'd recommend is tackling that car loan. Since you're paying ₹25,000 a month, let's explore if we can refinance it for a better interest rate, potentially freeing up some cash. Next, let's focus on building an emergency fund. Aim for at least 3-6 months' worth of essential living expenses – this will give you a safety net for unexpected situations. Even setting aside a small, consistent amount each month will make a difference. Once we have that emergency fund in place, we can start thinking about investments. Given your age, we have time to consider options like mutual funds or ETFs that align with your risk tolerance, which we'll need to determine. Finally, let's define your financial goals, like buying a home or planning for retirement. Knowing what you're working towards will h

In [6]:
suggestions_prompt = """
You are a **professional financial advisor** who provides personalized recommendations based on a user’s long-term financial profile and their current situation.

You are given:
1. **FinanceProfile** — a structured JSON object with the user’s detailed long-term financial data.
2. **Profile Summary** — a short narrative summarizing the user’s situation.

Your task:
Use the given data to create a **personalized recommendation** as if you were speaking directly to the user.
Do **not** repeat back all the facts they already know — instead, focus on **what they should do next** to improve their financial health.
Make the advice **specific to them** based on their profile, debt levels, savings, income, location, and goals.
Use a **supportive, encouraging, and human tone** that makes them feel understood and motivated.
Focus on **clear, practical steps** they can take, including:
  - Strategies to grow income.
  - Smart ways to manage and optimize expenses.
  - How to build or strengthen their emergency fund.
  - The best way to handle debts in their situation.
  - Opportunities to start or improve investments.
  - Long-term planning for goals like buying a home, retirement, or education.
Avoid sounding robotic or overly formal keep it conversational.
Write that feels like a friendly one-on-one chat.

Just provide the suggestion based on the data try best to provide the suggestion.

Not to include:
You should not include that we can think about the we can invest in share market but there is some details i want. 
example 
Considering your age, we have plenty of time to grow your wealth, and we can explore options like mutual funds or ETFs that align with your risk tolerance. Finally, let's also think about your broader goals. Do you have any plans for buying a home, retirement, or other major purchases? Understanding your goals will help us tailor a plan that works best for you.

This kind of question are strictly prohibited.
------

Give only the smart and based on the data suggestion.
---

Input:
{
  "FinanceProfile": {
    "user_age_years": 33,
    "user_occupation_job": "Account Manager",
    "user_location_city_country": "Ahmedabad, India",
    "user_has_spouse_partner": null,
    "user_number_of_children": null,
    "income_total_yearly_amount": 1400000.0,
    "income_monthly_take_home": 116666.67,
    "income_from_salary": null,
    "income_from_business": null,
    "income_from_investments": null,
    "income_from_rental": null,
    "income_other_sources": null,
    "expense_total_monthly": 3825.0,
    "expense_housing_rent_mortgage": null,
    "expense_food_groceries_dining": null,
    "expense_transportation_car_gas": null,
    "expense_utilities_bills": null,
    "expense_entertainment_hobbies": null,
    "expense_other_categories": null,
    "savings_monthly_amount": null,
    "savings_emergency_fund_total": null,
    "savings_percentage_of_income": null,
    "goal_buy_house_amount_needed": null,
    "goal_buy_house_timeline_years": null,
    "goal_retirement_target_amount": null,
    "goal_retirement_age_target": null,
    "goal_children_education_amount": null,
    "goal_vacation_travel_budget": null,
    "goal_other_major_purchases": null,
    "goal_most_important_priority": null,
    "investment_total_portfolio_value": null,
    "investment_stocks_value": null,
    "investment_bonds_value": null,
    "investment_mutual_funds_value": null,
    "investment_etf_value": null,
    "investment_crypto_value": null,
    "investment_real_estate_value": null,
    "investment_gold_precious_metals": null,
    "investment_fixed_deposits_value": null,
    "investment_retirement_401k_ira": null,
    "debt_total_amount_owed": 1150000.0,
    "debt_home_loan_mortgage": null,
    "debt_car_loan_amount": 1200000.0,
    "debt_student_loan_amount": null,
    "debt_credit_card_balance": null,
    "debt_personal_loan_amount": null,
    "debt_monthly_total_payments": 25000.0,
    "risk_tolerance_level": null,
    "risk_can_afford_to_lose_amount": null,
    "risk_investment_experience_years": null,
    "risk_reaction_to_market_drop": null,
    "prefer_stocks_over_bonds": null,
    "prefer_domestic_over_international": null,
    "prefer_growth_over_dividend": null,
    "prefer_individual_stocks_over_funds": null,
    "prefer_active_over_passive": null,
    "interested_in_crypto": null,
    "interested_in_real_estate": null,
    "avoid_specific_sectors": null,
    "tax_bracket_percentage": null,
    "tax_filing_status": null,
    "tax_state_residence": "Gujarat",
    "tax_deductions_claimed": null,
    "insurance_has_life": null,
    "insurance_has_health": null,
    "insurance_has_disability": null,
    "insurance_has_home_renters": null,
    "insurance_has_auto": null,
    "knowledge_understands_stocks": null,
    "knowledge_understands_bonds": null,
    "knowledge_understands_mutual_funds": null,
    "knowledge_needs_basic_education": null,
    "knowledge_confident_level": null,
    "history_best_investment_made": null,
    "history_worst_investment_made": null,
    "history_learned_lessons": null,
    "history_years_investing": null,
    "plans_major_purchase_next_year": null,
    "plans_career_change_expected": null,
    "plans_expecting_inheritance": null,
    "plans_starting_business": null,
    "prefers_simple_explanations": null,
    "prefers_detailed_analysis": null,
    "prefers_conservative_advice": null,
    "prefers_aggressive_strategies": null,
    "profile_last_updated_date": null,
    "portfolio_last_reviewed_date": null,
    "goals_last_discussed_date": null,
    "notes_special_circumstances": null,
    "notes_specific_questions_asked": null,
    "notes_advice_given_previously": null,
    "monthly_loan_payment": 25000.0,
    "available_credit": 125000.75
  }
  }

Profile Summary:
{  
  "profile_summary": "Neha Patel is a 33-year-old Account Manager residing in Ahmedabad, India. Her estimated annual income is approximately ₹1,400,000, with a monthly take-home of around ₹116,666.67. Her monthly expenses are estimated to be ₹3,825. She has an active car loan with a total amount owed of approximately ₹1,150,000 and makes monthly payments of ₹25,000. She has a current account with a balance of ₹125,000.75. There is no information available regarding her savings, investments, or specific financial goals. Further information is needed to create a comprehensive financial plan."

}

Output:
Give me the bullet point wise two to three clear suggestions.

example
Your entertainment expenses are much if possible can you reduce it.
Great Portfolio you can think to invest in the share market.

Give clear, short and human can think and take in action provide that kind of suggestions.

"""
